In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/MIMIC_IV_Pipeline/radiology.csv.gz', compression='gzip',nrows=200)
df.tail()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/MIMIC_IV_Pipeline/discharge.csv.gz'

In [ ]:
!pip -q install -U transformers torch accelerate --progress-bar off

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [ ]:
TEXT_COL = "text"
NOTE_ID_COL = "note_id"

df[TEXT_COL] = df[TEXT_COL].fillna("").astype(str)

In [ ]:
#using the model https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT

MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
def mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

def embed_text(text, max_len=512, stride=64):
    #usse the long texts and chunking them --> caluculating the avg of the embeddings
    enc = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=max_len,
        stride=stride,
        return_overflowing_tokens=True,
        return_attention_mask=True,
        return_tensors="pt"
    )

    chunk_embs = []
    for i in range(len(enc["input_ids"])):
        input_ids = enc["input_ids"][i].unsqueeze(0).to(device)
        attn_mask = enc["attention_mask"][i].unsqueeze(0).to(device)

        with torch.no_grad():
            out = model(input_ids=input_ids, attention_mask=attn_mask)
            emb = mean_pool(out.last_hidden_state, attn_mask)  # (1,768)
            chunk_embs.append(emb[0])

    note_emb = torch.stack(chunk_embs).mean(dim=0)  # average chunks
    note_emb = torch.nn.functional.normalize(note_emb, p=2, dim=0)  # normalize
    return note_emb.cpu().tolist()

#run the embed func through the texts
all_embs = []
for text in tqdm(df[TEXT_COL].tolist(), desc="Embedding notes"):
    all_embs.append(embed_text(text))

df["text_embed"] = all_embs

Embedding notes: 100%|██████████| 200/200 [49:10<00:00, 14.75s/it]


In [ ]:
out_path = "/content/drive/MyDrive/MIMIC_IV_Pipeline/discharge_with_clinicalbert_embedding.csv"
df.to_csv(out_path, index=False)
print(f"Saved embeddings → {out_path}")
print(df[[NOTE_ID_COL, "text_embed"]].head(2))

Saved embeddings → /content/drive/MyDrive/MIMIC_IV_Pipeline/discharge_with_clinicalbert_embedding.csv
          note_id                                         text_embed
0  10000032-DS-21  [0.004929497372359037, -0.014205681160092354, ...
1  10000032-DS-22  [0.0034717735834419727, -0.013519076630473137,...


In [ ]:
df.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,text_embed
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...,"[0.004929497372359037, -0.014205681160092354, ..."
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...,"[0.0034717735834419727, -0.013519076630473137,..."
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...,"[0.00036219815956428647, -0.012180660851299763..."
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...,"[0.0006128204986453056, -0.015201750211417675,..."
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...,"[-0.0037114580627530813, -0.019275402650237083..."
